# 使用Left-Leaning RBTree实现的映射

- 参考资料
  - <<算法>>(第四版) 3.3小节
  - https://www.cs.princeton.edu/~rs/talks/LLRB/RedBlack.pdf
  - https://www.cs.princeton.edu/~rs/talks/LLRB/LLRB.pdf

In [1]:
RED = True
BLACK = False

In [2]:
class _Node(object):
    l = None
    r = None

    def __init__(self, k, v, n, color):
        self.k = k
        self.v = v
        self.n = n
        self.color = color

In [3]:
class RBTreeMapping(object):
    """使用 LLRBT 实现的映射

    .. note::
       * 下文中 ``\\`` 或  ``//`` 均表示一个红连接
    """

    root = None

    def size(self):
        return self._size(self.root)

    @classmethod
    def _size(cls, node):
        return 0 if node is None else node.n

    def get(self, k):
        node = cls._get(self.root, k)
        return node.v if node else None

    @classmethod
    def _get(cls, node, k):
        if not node:     return None
        if k == node.k:  return node
        elif k < node.k: return cls._get(node.l, k)
        else:            return cls._get(node.r, k)

    @classmethod
    def _is_red(object, node):
        """是否为红色
        """
        if node is None: return False
        return node.color == RED

    @classmethod
    def _rotate_left(cls , node):
        """左旋

          |                |
          A                B
         / \\    ==>     // \
        1   B           A    3
           / \         / \
          2   3       1   2
        """
        x = node.r
        node.r = x.l
        x.l = node
        x.color = node.color
        node.color = RED
        x.n = node.n
        node.n = cls._size(node.l) + \
                 cls._size(node.r) + 1
        return x

    @classmethod
    def _rotate_right(cls, node):
        """右旋
        """
        x = node.l
        node.l = x.r
        x.r = node
        x.color = node.color
        node.color = RED
        x.n = node.n
        node.n = cls._size(node.l) + \
                 cls._size(node.r) + 1
        return x

    @classmethod
    def _flip_colors(cls, node):
        """翻转节点颜色
        """
        node.color = not node.color
        node.l.color = not node.l.color
        node.r.color = not node.r.color

    @classmethod
    def _fix_up(cls, node):
        """通过旋转和变色使树保持LLRBT的性质
        """
        # 红连接永远保持 leaning-left
        if cls._is_red(node.r) and not cls._is_red(node.l):
            node = cls._rotate_left(node)

        # 将连续的红连接右旋, 得到一个4-node节点
        if cls._is_red(node.l) and cls._is_red(node.l.l):
            node = cls._rotate_right(node)

        # 如果是4-node节点, 通过变色分裂之
        if cls._is_red(node.r) and cls._is_red(node.l):
            cls._flip_colors(node)

        node.n = cls._size(node.l) + cls._size(node.r) + 1
        return node

    def put(self, k, v):
        self.root = self._put(self.root, k, v)
        # 旋转和变色可能导致根节点变成 ``RED``, 将其置为 ``BLACK``
        # 另: 每当根节点变色一次, 意味着树的高度增加了 1
        self.root.color = BLACK

    @classmethod
    def _put(cls, node, k, v):
        """
        插入的节点默认是 ``RED``, 从下图可以看出, 为了保持LLRBT的性质,
        我们通过旋转和变色, 将新插入的红连接 **从下往上** 进行了传递.

               insert 1️⃣     |     insert 2️⃣
              ----------     ○     ----------
              |             //\             |
              |            ○                |
              |              ↓              |
              |              | insert 3️⃣    ↓
              |              ○
              |             //\             |    flip-color    ||
              |            ○                ○    ---------->   ○
              |           /\\             // \\      6️⃣       / \ 
              ↓              ○           ○     ○             ○   ○
                             ↓              ↑
              |     l-rotate |4️⃣            |
              ○    ←---------|              |
             //\                            |
            ○             r-rotate 5️⃣       |
           //\     --------------------------
          ○
        """
        # 新插入的节总是红色
        if node is None: return _Node(k, v, 1, RED)

        if k > node.k:   node.r = cls._put(node.r, k, v)
        elif k < node.k: node.l = cls._put(node.l, k, v)
        else:            node.v = v

        return cls._fix_up(node)

    def max(self):
        if self.is_empty(): return
        return self._max(self.root).k

    @classmethod
    def _max(cls, node):
        if node.r is None: return node
        else: return cls._max(node.r)

    def delete_max(self):
        if self.is_empty(): return
        self.root = self._delete_max(self.root)
        if not self.is_empty():
            self.root.color = BLACK

    @classmethod
    def _delete_max(cls, node):
        # 1. 该逻辑必须在 2 之前执行, 否则若 ``X.l`` 是 ``RED``,
        # 则删除 ``X`` 后的同时, 相当于也把 ``X.l`` 也删除了.
        # 也即须保证 ``X`` 是在 3-nodes or 4-nodes 的最右边!
        #
        # CASE 1: ``X.l`` 是 ``RED``(由 3 可保证此时 ``X`` 是 ``BLACK``), 做右旋操作
        #    |
        #    X
        #  // \
        # ○
        #
        # CASE 2: ``X`` 是 ``RED``(由 3 可保证此时 ``X`` 是 ``RED``), 不需要右旋操作
        #    ||
        #    X
        #   / \
        if cls._is_red(node.l):
            node = cls._rotate_right(node)

        # 2. 删除节点
        if node.r is None: return None

        # 3. 从上往下传递 ``RED``, 确保要被删除的节点在 3-nodes or 4-nodes 中
        if not cls._is_red(node.r) and not cls._is_red(node.r.l):
            node = cls._move_red_right(node)

        # 4. 从上往下递归删除
        node.r = cls._delete_max(node.r)

        # 5. 从下往上递归时, 修复
        return cls._fix_up(node)

    @classmethod
    def _move_red_right(cls, node):
        """
        确保要删除的节点 ``X`` 在一个 3-nodes or 4-nodes 中

        CASE 1: ``node.l.l`` 是 ``BLACK``
            |              |
            ○     ===>     ○
           / \           // \\
          ○   X         ○     X
         /   / \       /     / \

        CASE 2: ``node.l.l`` 是 ``RED``
            |             |             |             |
            ○             ○             ○             ○
           / \    1️⃣    // \\    2️⃣   // \\   3️⃣    /  \
          ○   X  ===>  ○     X  ===> ○     ○  ===>  ○   ○
         //  / \      //    / \            \\            \\
        ○            ○                       X            X
        """
        cls._flip_colors(node)               # 1️⃣
        if cls._is_red(node.l.l):
            node = cls._rotate_right(node)   # 2️⃣
            cls._flip_colors(node)           # 3️⃣
        return node

    def min(self):
        if self.is_empty(): return
        return self._min(self.root).k

    @classmethod
    def _min(cls, node):
        if node.l is None: return node
        else: return cls._min(node.l)

    def delete_min(self):
        if self.is_empty(): return
        self.root = self._delete_min(self.root)
        if not self.is_empty():
            self.root.color = BLACK

    @classmethod
    def _delete_min(cls, node):
        if node.l is None: return None

        if not cls._is_red(node.l) and not cls._is_red(node.l.l):
            node = cls._move_red_left(node)

        node.l = cls._delete_min(node.l)
        return cls._fix_up(node)

    @classmethod
    def _move_red_left(cls, node):
        """确保要删除的节点 ``X`` 在 3-nodes or 4-nodes 中

        CASE 1: ``node.r.l`` 是 ``BLACK``
            |                |
            ○                ○
           / \     ====>   // \\
          X   ○           X    ○
         /   / \         /    / \

        CASE 2: ``node.r.l`` 是 ``RED``
            |                |                |
            ○       1️⃣       ○       2️⃣       ○
           / \     ====>   // \\    ====>   // \\
          X   ○           X    ○           X     ○
         /  // \         /   // \         /      \\

         3️⃣     |      4️⃣     ||
        ====>   ○     ====>   ○
              // \\          / \
             ○              ○
            //             //
           X              X
        """
        cls._flip_colors(node)                    # 1️⃣
        if cls._is_red(node.r.l):
            node.r = cls._rotate_right(node.r)    # 2️⃣
            node = cls._rotate_left(node)         # 3️⃣
            cls._flip_colors(node)                # 4️⃣
        return node

    def delete(self, k):
        # 先判断 ``k`` 是否存在, 不然 ``_delete`` 中的逻辑会复杂很多
        if self.get(k) is None: return
        self.root = self._delete(self.root, k)
        if not self.is_empty():
            self.root.color = BLACK

    @classmethod
    def _delete(cls, node, k):
        if k < node.k:
            # 通过 ``_move_red_left`` 确保 ``node.l`` 在 3-nodes or 4-nodes 中
            if not cls._is_red(node.l) and not cls._is_red(node.l.l):
                node = cls._move_red_left(node)
            node.l = cls._delete(node.l, k)
        else:
            if cls._is_red(node.l):
                node = cls._rotate_right(node)

            # 在树的底部删除节点
            if k == node.k and node.r is None:
                return None

            # 通过 ``_move_red_right`` 确保 ``node.r`` 在 3-nodes or 4-nodes 中
            if not cls._is_red(node.r) and not cls._is_red(node.r.l):
                node = cls._move_red_right(node)

            if k == node.k:
                # 在树的非底部删除节点, 用 ``node`` 的后继节点
                # 替代当前节点, 然后删除后继节点即可
                successor = cls._min(node.r)
                node.k = successor.k
                node.v = successor.v
                node.r = cls._delete_min(node.r)
            else:
                node.r = cls._delete(node.r, k)

        return cls._fix_up(node)
